In [1]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image

Using TensorFlow backend.


# Base Model

In [13]:
if True:
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), name= 'conv1_1'))
    model.add(Activation('relu', name='relu1_1'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), name= 'conv1_2'))
    model.add(Activation('relu', name='relu1_2'))
    model.add(MaxPooling2D((2,2), strides=(2,2), name='pool1'))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), name= 'conv2_1'))
    model.add(Activation('relu', name='relu2_1'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), name= 'conv2_2'))
    model.add(Activation('relu', name='relu2_2'))
    model.add(MaxPooling2D((2,2), strides=(2,2), name='pool2'))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), name= 'conv3_1'))
    model.add(Activation('relu', name='relu3_1'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), name= 'conv3_2'))
    model.add(Activation('relu', name='relu3_2'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), name= 'conv3_3'))
    model.add(Activation('relu', name='relu3_3'))
    model.add(MaxPooling2D((2,2), strides=(2,2), name='pool3'))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), name= 'conv4_1'))
    model.add(Activation('relu', name='relu4_1'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), name= 'conv4_2'))
    model.add(Activation('relu', name='relu4_2'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), name= 'conv4_3'))
    model.add(Activation('relu', name='relu4_3'))
    model.add(MaxPooling2D((2,2), strides=(2,2), name='pool4'))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), name= 'conv5_1'))
    model.add(Activation('relu', name='relu5_1'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), name= 'conv5_2'))
    model.add(Activation('relu', name='relu5_2'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), name= 'conv5_3'))
    model.add(Activation('relu', name='relu5_3'))
    model.add(MaxPooling2D((2,2), strides=(2,2), name='pool5'))

    model.add(Convolution2D(4096, (7, 7), name= 'fc6'))
    model.add(Activation('relu', name='relu6'))
    model.add(Dropout(0.5, name='dropout6'))
    model.add(Convolution2D(4096, (1, 1), name= 'fc7'))
    model.add(Activation('relu', name='relu7'))
    model.add(Dropout(0.5, name='dropout7'))
    model.add(Convolution2D(2622, (1, 1), name= 'fc8'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Activation('softmax', name= 'softmax'))

# MatConvNet to Keras

In [4]:
from scipy.io import loadmat

In [5]:
data = loadmat('vgg_face.mat', matlab_compatible=False, struct_as_record=False)

In [6]:
data

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Oct 13 16:54:01 2015',
 '__version__': '1.0',
 'net': array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x00000208ACA1D1D0>]],
       dtype=object)}

In [7]:
net = data['net'][0][0]

In [ ]:
net.

In [8]:
ref_model_layers = net.layers

In [9]:
ref_model_layers.shape

(1, 39)

In [10]:
ref_model_layers = ref_model_layers[0]

In [23]:
for layer in ref_model_layers:
    print(layer[0][0].name)

['conv1_1']
['relu1_1']
['conv1_2']
['relu1_2']
['pool1']
['conv2_1']
['relu2_1']
['conv2_2']
['relu2_2']
['pool2']
['conv3_1']
['relu3_1']
['conv3_2']
['relu3_2']
['conv3_3']
['relu3_3']
['pool3']
['conv4_1']
['relu4_1']
['conv4_2']
['relu4_2']
['conv4_3']
['relu4_3']
['pool4']
['conv5_1']
['relu5_1']
['conv5_2']
['relu5_2']
['conv5_3']
['relu5_3']
['pool5']
['fc6']
['relu6']
['dropout6']
['fc7']
['relu7']
['dropout7']
['fc8']
['softmax']


In [11]:
num_of_ref_model_layers = ref_model_layers.shape[0]

In [14]:
base_model_layer_names = [layer.name for layer in model.layers]

In [28]:
for layer in model.layers:
    layer_name = layer.name
    try:
        print(layer_name,": ",layer.weights[0].shape)
    except:
        print("",end='')
        #print(layer_name)

conv1_1 :  (3, 3, 3, 64)
conv1_2 :  (3, 3, 64, 64)
conv2_1 :  (3, 3, 64, 128)
conv2_2 :  (3, 3, 128, 128)
conv3_1 :  (3, 3, 128, 256)
conv3_2 :  (3, 3, 256, 256)
conv3_3 :  (3, 3, 256, 256)
conv4_1 :  (3, 3, 256, 512)
conv4_2 :  (3, 3, 512, 512)
conv4_3 :  (3, 3, 512, 512)
conv5_1 :  (3, 3, 512, 512)
conv5_2 :  (3, 3, 512, 512)
conv5_3 :  (3, 3, 512, 512)
fc6 :  (7, 7, 512, 4096)
fc7 :  (1, 1, 4096, 4096)
fc8 :  (1, 1, 4096, 2622)


In [30]:
for i in range(num_of_ref_model_layers):
    ref_model_layer = ref_model_layers[i][0,0].name[0]
    
    try:
        weights = ref_model_layers[i][0,0].weights[0,0]
        print(ref_model_layer,": ",weights.shape)
    except:
        #print(ref_model_layer)
        print("",end='')

conv1_1 :  (3, 3, 3, 64)
conv1_2 :  (3, 3, 64, 64)
conv2_1 :  (3, 3, 64, 128)
conv2_2 :  (3, 3, 128, 128)
conv3_1 :  (3, 3, 128, 256)
conv3_2 :  (3, 3, 256, 256)
conv3_3 :  (3, 3, 256, 256)
conv4_1 :  (3, 3, 256, 512)
conv4_2 :  (3, 3, 512, 512)
conv4_3 :  (3, 3, 512, 512)
conv5_1 :  (3, 3, 512, 512)
conv5_2 :  (3, 3, 512, 512)
conv5_3 :  (3, 3, 512, 512)
fc6 :  (7, 7, 512, 4096)
fc7 :  (1, 1, 4096, 4096)
fc8 :  (1, 1, 4096, 2622)


In [31]:
for i in range(num_of_ref_model_layers):
    ref_model_layer = ref_model_layers[i][0,0].name[0]
    if ref_model_layer in base_model_layer_names:
        #we just need to set convolution and fully connected weights
        if ref_model_layer.find("conv") == 0 or ref_model_layer.find("fc") == 0:
            print(i,". ",ref_model_layer)
            base_model_index = base_model_layer_names.index(ref_model_layer)
            
            weights = ref_model_layers[i][0,0].weights[0,0]
            bias = ref_model_layers[i][0,0].weights[0,1]
            
            model.layers[base_model_index].set_weights([weights, bias[:,0]])

0 .  conv1_1
2 .  conv1_2
5 .  conv2_1
7 .  conv2_2
10 .  conv3_1
12 .  conv3_2
14 .  conv3_3
17 .  conv4_1
19 .  conv4_2
21 .  conv4_3
24 .  conv5_1
26 .  conv5_2
28 .  conv5_3
31 .  fc6
34 .  fc7
37 .  fc8


In [32]:
model.save_weights('vgg-face-weights.h5')

# Constructing reference model with clues

In [ ]:
model = Sequential()

for i in range(num_of_ref_model_layers):
    ref_model_layer = ref_model_layers[i][0][0].name[0]
    if ref_model_layer.find("conv") == 0 or ref_model_layer.find("fc") == 0:
        weights = ref_model_layers[i][0,0].weights
        weights_shape = weights[0][0].shape
        #print(":", weights_shape)
        filter_x = weights_shape[0]; filter_y = weights_shape[1]
        number_of_filters = weights_shape[3]
        
        if ref_model_layer.find("conv") == 0:
            print("ZeroPadding2D((1,1))")
            if i == 0:
                model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
            else:
                model.add(ZeroPadding2D((1,1)))
        
        print("Convolution2D(",number_of_filters,", (",filter_x,", ",filter_y,"), name='",ref_model_layer,"')")
        model.add(Convolution2D(number_of_filters, (filter_x, filter_y), name= ref_model_layer))
        
    else:
        if ref_model_layer.find("relu") == 0:
            print("Activation('relu', name=",ref_model_layer)
            model.add(Activation('relu', name=ref_model_layer))
        elif ref_model_layer.find("dropout") == 0:
            print("Dropout(0.5, name=",ref_model_layer,")")
            model.add(Dropout(0.5, name=ref_model_layer))
        elif ref_model_layer.find("pool") == 0:
            print("MaxPooling2D((2,2), strides=(2,2), name=",ref_model_layer,")")
            model.add(MaxPooling2D((2,2), strides=(2,2), name=ref_model_layer))
        elif ref_model_layer.find("softmax") == 0:
            print("Activation('softmax', name=",ref_model_layer,")")
            model.add(Activation('softmax', name=ref_model_layer))
        else:
            print("-->",ref_model_layer)